# Project Ruby - Baseline

## Goals: 
1. Predict how many comments will follow a given comment.
2. Generate comments that will generate most children comments.

In [1]:
import json

def read_dataset():
    comments = []
    features = ['body', 'author', 'subreddit', 'id', 'parent_id', 'score', 'controversiality', 'link_id']
    subreddits = ['news']
    comments_size = 100000
    years = ["2017"]
    months = ["01"]
    for year in years:
        for month in months:
            filename = "reddit/" + year + "/RC_" + year + "-" + month
            with open(filename, "r") as data:
                for line in data:
                    if len(comments) < comments_size:
                        comment_data = json.loads(line)
                        if comment_data['subreddit'] in subreddits:
                            comment = dict()
                            for feature in features:
                                comment[feature] = comment_data[feature]
                            comments.append(comment)
                    else:
                        break
    print ("Example Comment:")
    print (comments[0])
    return comments

comments = read_dataset()

Example Comment:
{'body': "'Cause your dick might hit a butt and make baby Jesus cry. ", 'author': 'Number6isNo1', 'subreddit': 'news', 'id': 'dbumnrk', 'parent_id': 't1_dbumi82', 'score': 26, 'controversiality': 0, 'link_id': 't3_5lbkgm'}


# Baseline 1: Logistic Regression with Unigram Features

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

def build_vocabulary(comments):
    vocabulary = set()
    for comment in comments:
        for word in comment:
            vocabulary.add(word)
    return vocabulary

def prepare_train(comments, vocabulary, train_size):
    train = comments[:int(train_size * len(comments))]
    test = comments[len(comments)-len(train):]
    vocabulary = list(vocabulary)
    cv = CountVectorizer(vocabulary=vocabulary)
    
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []

    for comment in train:
        train_X.append(comment['body'])
        train_Y.append(comment['score'])
    train_X = cv.fit_transform(train_X)

    for comment in test:
        test_X.append(comment['body'])
        test_Y.append(comment['score'])
    test_X = cv.fit_transform(test_X)
    
    return train_X, train_Y, test_X, test_Y

vocabulary = build_vocabulary(comments)
train_X, train_Y, test_X, test_Y = prepare_train(comments, vocabulary, train_size=0.9)

In [ ]:
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import numpy as np

lr = LinearRegression()
lr = lr.fit(train_X, train_Y)
print (lr.score(train_X, train_Y))
prediction = lr.predict(test_X)
# prediction = np.zeros(len(test_Y))
print (metrics.mean_absolute_error(test_Y, prediction))
print (metrics.mean_squared_error(test_Y, prediction))

6.3401532695e-06
21.3334794347
34704.3763216


# Baseline 2: Neural Network Regression with GloVe

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.txt'
word2vec_output_file = 'word2vec.txt'
model = glove2word2vec(glove_input_file, word2vec_output_file)

from gensim.models import KeyedVectors
filename = 'word2vec.txt'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
import numpy as np

def prepare_train(comments, train_size):
    train = comments[:int(train_size * len(comments))]
    test = comments[len(comments)-len(train):]
    wordvec_dim = model['test'].shape
    
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []

    for comment in train:
        sentence_vector = np.zeros(wordvec_dim)
        count = 0
        for word in comment['body']:
            if word in model.vocab:
                count += 1
                sentence_vector += model[word]
        sentence_vector /= count
        train_X.append(sentence_vector)
        train_Y.append(comment['score'])

    for comment in test:
        sentence_vector = np.zeros(wordvec_dim)
        count = 0
        for word in comment['body']:
            if word in model.vocab:
                count += 1
                sentence_vector += model[word]
        sentence_vector /= count
        test_X.append(sentence_vector)
        test_Y.append(comment['score'])
    
    return train_X, train_Y, test_X, test_Y

train_X, train_Y, test_X, test_Y = prepare_train(comments, train_size=0.9)

In [ ]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt

nn = MLPRegressor(
    hidden_layer_sizes=(10,),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=9, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
nn.fit(train_X, train_Y)
prediction = nn.predict(test_X)
print (nn.score(train_X, train_Y))
print (metrics.mean_absolute_error(test_Y, prediction))
print (metrics.mean_squared_error(test_Y, prediction))